In [2]:
#Lesum inn gögnin
import numpy as np

gogn = np.loadtxt("Gögn/games.csv", delimiter=",", dtype=str, comments=None)
#display(gogn)
gogn.shape

(20059, 16)

In [3]:
#Eyðum gögnum sem eru ekki notuð
dalkarTilAdEyda = [15,14,13,10,8,7,5,3,2,1]
gogn = np.transpose(gogn)
gogn = np.delete(gogn,dalkarTilAdEyda,0)
gogn = np.transpose(gogn)
gogn.shape

(20059, 6)

In [4]:
#Finnum lengd lengstu skákar
import numpy as np
max_lengd = 0
temp_max_lengd = 0
maxi = 0
for i in range(len(gogn[:,0])):
    leikir = gogn[i,5].split()
    temp_max_lengd = len(leikir)
    if temp_max_lengd > max_lengd:
        max_lengd = temp_max_lengd
        maxi = i
print(max_lengd)
print(maxi)

#Setjum skákirnar í fylki, leik fyrir leik
leikir = np.zeros((len(gogn[:,0])-1,max_lengd), dtype=list)
for i in range(1, len(gogn[:,0])):
    temp_leikir = gogn[i,5].split()
    for j in range(len(temp_leikir)):
        leikir[i-1,j] = temp_leikir[j]
print(leikir.shape)

349
11555
(20058, 349)


In [5]:
#Breytum uppsetingu á leikjum úr (d4 d5...) í bitmap og bætum við númeri leiks
import chess
leikir_fen = np.zeros((len(gogn[:,0])-1,max_lengd), dtype=list)
for i in range(len(leikir[:,0])):
    for j in range(len(leikir[0,:])):
        if leikir[i,j] != 0:
            board = chess.Board()
            for k in range(j):
                board.push_san(leikir[i,k])
            leikir_fen[i,j] = board.fen()
        else:
            break

#board = chess.Board()
#board.push_san('d4')
#board.fen

In [11]:
np.savetxt("leikir_fen.csv", leikir_fen, delimiter=",", fmt="%s")

In [72]:
#Finnum evaluation fyrir hvern leik með Stockfish
from stockfish import Stockfish

stockfish = Stockfish(path="/home/magnus/Downloads/Installers of setup/stockfish_15.1_linux_x64/stockfish-ubuntu-20.04-x86-64")
stockfish.update_engine_parameters({"Threads": 10, "Minimum Thinking Time": 0, "Hash": 32})
stockfish.set_depth(10)
leikir_eval = np.zeros((len(gogn[:,0])-1,max_lengd), dtype=list)
for i in range(len(leikir_fen[:,0])):
#for i in range(100):
    for j in range(len(leikir_fen[0,:])):
        if leikir_fen[i,j] != 0:
            stockfish.set_fen_position(leikir_fen[i,j])
            a = stockfish.get_evaluation()
            leikir_eval[i,j] = a['value']
        else:
            break


In [63]:
stockfish.depth

'15'

In [55]:
from stockfish import Stockfish
stockfish = Stockfish(path="/home/magnus/Downloads/Installers of setup/stockfish_15.1_linux_x64/stockfish-ubuntu-20.04-x86-64")
stockfish.get_parameters()

{'Debug Log File': '',
 'Contempt': 0,
 'Min Split Depth': 0,
 'Ponder': 'false',
 'MultiPV': 1,
 'Skill Level': 20,
 'Move Overhead': 10,
 'Minimum Thinking Time': 20,
 'Slow Mover': 100,
 'UCI_Chess960': 'false',
 'UCI_LimitStrength': 'false',
 'UCI_Elo': 1350,
 'Threads': 1,
 'Hash': 16}

In [48]:
stockfish.get_stockfish_major_version()

14